# Activity Log - View

**Author**: Melissa Coates  

**Last updated and tested**: August 22, 2022 

**Purpose:** This notebook contains multiple types of ad hoc scripts to view the [Power BI activity log](https://docs.microsoft.com/en-us/power-bi/admin/service-admin-auditing):

- View activity log: one type of activity, for one day
- View activity log: all activities, for one user, for one day

**Type of authentication**: You can log in with either:

- A domain user account (which Power BI administrator permissions), or
- A service principal (with permission to run admin APIs)

**How data is being accessed:** The notebook uses:

- Power BI Management Module for authentication: Connect-PowerBIServiceAccount
- Power BI Management Module for extracting the activity log data: Get-PowerBIActivityEvent

**Type of script**: Interactive, ad hoc (non-scheduled). This notebook is a guided learning experience to get familiar with how to query the activity log. It's not production-ready.

This notebook is not designed to do a 'Run all.' Rather, it's designed to be run cell-by-cell, based on the scenario. More information, including prerequisites, is included at the bottom of this notebook.

## **Log in to the Power BI Service**
---
**Update before executing:**

- **Type of authentication:** 
    - **Line 2 - Choice for type of auth:** Input either '**User**' or '**SP**' without the quotes. This tells the script whether or not to log in as a user or a service principal.
- **If using user authentication:**
    - **Line 3 - Email address:** Input the email address for the user. This user must be a Power BI administrator. MFA can't be enabled for this account.
- **If using service principal authentication:**
    - **Line 4 - Azure AD app ID**: Input the app ID (aka client ID). 
    - **Line 5 - Azure tenant ID**: Input the tenant ID (aka directory ID).

**Interactive prompts:**

The script will interactively prompt you for the sensitive information (because hard-coding of passwords and secrets isn't secure):

- **If using user authentication:**
    - It will ask for the user's password.
- **If using service principal authentication:**
    - It will ask for the app secret.

**Comments:**

- This script uses the PowerShell cmdlet: [<u>Connect-PowerBIServiceAccount</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.profile/connect-powerbiserviceaccount).
- After signing in, the access token is active for one hour.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$TypeOfAuth = 'User'  #Options: User or SP
[string]$DomainUserEmailAddr = 'user@domain.com'
[string]$AzureADAppID = '1234-1234'
[string]$AzureTenantID = '1234-1234'
#--------------------------------------------

if ($TypeOfAuth -eq 'User')
{
    Write-Verbose "Domain user authentication will be used." -Verbose
    [securestring]$DomainUserPW = Read-Host -Prompt "Input password for $DomainUserEmailAddr" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($DomainUserEmailAddr, $DomainUserPW)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj
}
elseif ($TypeOfAuth -eq 'SP')
{
    Write-Verbose "Service principal authentication will be used." -Verbose
    [securestring]$AzureADAppSecret = Read-Host -Prompt "Input secret for $AzureADAppID" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($AzureADAppID, $AzureADAppSecret)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj -ServicePrincipal -Tenant $AzureTenantID
}
else 
{
    Write-Verbose "There's a problem. The Type of Auth variable input was $TypeOfAuth. Expected values are User or SP." -Verbose
}

## **View Activity Log: One Type of Activity, One Day**

* * *

The next script will allow you to view the raw data for one type of activity, for a specific day. This is useful when you're looking for information about a particular type of event.

**Update before executing:**

- **Line 2 -** **Name of the activity:** This script displays data for one specific event, such as the 'ShareReport' or 'ViewReport' events. Use the "Operation Name" from this [reference](https://docs.microsoft.com/en-us/power-bi/admin/service-admin-auditing#operations-available-in-the-audit-and-activity-logs).
- **Line 3 - **Date of the activity:** Format is YYYY-MM-DD. Data up to 30 days in the past may be retrieved.**

**Comments:**

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIActivityEvent</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.admin/get-powerbiactivityevent).
- **Time zones:** All activity events are based on UTC datetime.
- **Date range:** Within a single execution of this cmdlet, the start and end datetime must be within the same UTC day.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$ActivityType = 'ViewReport'
[string]$ActivityDate = '2022-08-20' #YYYY-MM-DD
#--------------------------------------------

Get-PowerBIActivityEvent -StartDateTime ($ActivityDate+'T00:00:00.000') -EndDateTime ($ActivityDate+'T23:59:59.999') -ActivityType $ActivityType

## **View Activity Log: All Activities, One User, One Day**
---
The next script will allow you to view all activities, for a specific user, for a specific day. This is useful when you're looking for information about a certain person.

**Update before executing:**

- **Line 2 - User who performed the activity:** This is the email address of the user who performed the activity.
- **Line 3 - Date of the activity:** Format is YYYY-MM-DD. Data up to 30 days in the past may be retrieved.

**Comments**:

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIActivityEvent</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.admin/get-powerbiactivityevent). 
- **Time zones:** All activity events are based on UTC datetime.
- **Date range:** Within a single execution of this cmdlet, the start and end datetime must be within the same UTC day.

In [ ]:
#-----------------INPUT AREA-----------------
[string]$UserWhoPerformedActivity = 'user@domain.com'
[string]$ActivityDate = '2022-08-20' #YYYY-MM-DD
#--------------------------------------------

Get-PowerBIActivityEvent -StartDateTime ($ActivityDate+'T00:00:00.000') -EndDateTime ($ActivityDate+'T23:59:59.999') -User $UserWhoPerformedActivity

## **More Information About This File**

* * *

## <span style="font-size:14px;">Prerequisites for running this script:</span>

- The [Power BI Management module](https://www.powershellgallery.com/packages/MicrosoftPowerBIMgmt) for PowerShell needs to be installed.
    
- The [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension needs to be installed and enabled in VS Code and/or Azure Data Studio.
    
- Either a user account with Power BI administrator permissions, or a service principal with permission to run admin APIs.
    

This script **interactively prompts** for the sensitive information to be passed into a credential object. Therefore, it's only useful for ad hoc purposes. A different technique is used for securely storing the credentials when a script is to be operationalized and scheduled.

The original / most recent version of this notebook can be found on Melissa Coates' [Power-BI-Admin repository in GitHub](https://github.com/sqlchick/Power-BI-Admin).